### Menadżer modelu w pigułce

Czym są menadżery?

Manager to interfejs udostępniający zapytania bazodanowe z poziomy modelu Django. Warto w tym miejscu zwrócić uwagę na to, że metody modelu są operacjami dotyczącymi pojedynczego wiersza tabeli (obiektu), podczas gdy metody managera są operacjami na całej tabeli (ew. na wielu wierszach).

In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'intro.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

Będziemy używali modelu z samouczka Django

<code>
from django.db import models

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
</code>

Do odpytywania modelu używamy jego managera (domyślnie umieszczonego w atrybucie `objects`).

In [ ]:
from orm_app.models import Question

result = Question.objects.filter(question_text="Is this looking ok?")
result

Ale równie dobrze możemy napisać:

<code>
from django.db import models

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
    objects = models.Manager()
</code>

otrzymamy ten sam efekt.

Ale modele mogą mieć więcej managerów. Jeżeli chcemy stworzyć własnego managera możemy to zrobić na kilka sposobów:
- `objects = CustomManager()` jeżeli mamy własnego managera
- `objects = CustomQuerySet.as_manager()` jeżeli mamy własnego queryseta
- `objects = CustomManager.from_queryset(CustomQuerySet)()` jeżeli mamy własnego managera i własnego queryseta

Popatrzmy na każdy ze sposobów.

#### 1. CustomManager

<code>
from django.db import models

class QuestionManager(models.Manager):
    ...

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
    objects = QuestionManager()
</code>

#### 2. CustomQuerySet

<code>
from django.db import models

class QuestionQuerySet(models.QuerySet):
    ...

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
    objects = QuestionQuerySet.as_manager()
</code>

#### 3. CustomManager i CustomQuerySet

<code>
from django.db import models

class QuestionQuerySet(models.QuerySet):
    ...

class QuestionManager(models.Manager):
    ...

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
    objects = QuestionManager.from_queryset(QuestionQuerySet)()
</code>

Ok, ale której z metod powinniśmy użyć. Managera, queryseta ? Jaka jest różnica między nimi?

Podstawowa różnica polega na umiejętności łańcuchowania. QuerySet posiada płynny interfejs (ang. fluent interface) i w związku z tym potrafi się łańcuchować podczas gdy manager tego nie potrafi. Czyli jeżeli chcemy, żeby nasza customowa metoda odpytywania mogła być łańcuchowana używamy queryseta, w przeciwnym razie możemy użyć managera.

<code>
from django.db import models

class QuestionManager(models.Manager):
    def published_in_future(self):
        return self.filter(pub_date__gt=timezone.now())

    def questions_about_me(self):
        return self.filter(question_text__icontains="Jan Kowalski")

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
    objects = QuestionManager()
</code>

I teraz

<code>
# to zadziała
Question.objects.published_in_future()
Question.object.questions_about_me()
# ale to już nie (AttributeError)
Question.objects.published_in_future().questions_about_me()
</code>

Jeżeli chcemy, żeby mogły się łańcuchować użyjmy queryseta.

<code>
from django.db import models

class QuestionQuerySet(models.QuerySet):
    def published_in_future(self):
        return self.filter(pub_date__gt=timezone.now())

    def questions_about_me(self):
        return self.filter(question_text__icontains="Jan Kowalski")

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
    objects = QuestionQuerySet.as_manager()
</code>

Teraz

<code>
# to wciąż zadziała
Question.objects.published_in_future()
Question.object.questions_about_me()
# a poza tym, teraz to też działa
Question.objects.published_in_future().questions_about_me()
</code>

Najprościej myśleć o tym w kategoriach crud.

Managera tworzymy jeżeli mamy jakieś customowe metody związane z:
- creating
- updating
- deleting

w pozostałych przypadkach takich jak:
- filtering
- annotations/aggregations
- reading

używamy queryseta.

Jeżeli mamy i to i to, wtedy wykorzystujemy trzecią metodą.

<code>
from django.db import models

class QuestionManager(models.Manager):
    def create_question(self, question: str): ...
        
class QuestionQuerySet(models.QuerySet):
    def published_in_future(self): ...
    def questions_about_me(self): ...

class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")
    objects = QuestionManager.from_queryset(QuestionQuerySet)()
</code>

Ale po co w ogóle używać własnych managerów ?

Powodów jest kilka:
- zwiększają czytelność kodu
- enkapsulują
- są proste w testowaniu
- są świetnym wstępem do zaawansowanych zagadnień ORM

Kiedy używamy?

Kiedy widzimy zapytania bazodanowe do przepisania/napisania, które prawdopodobnie będą się powtarzać w kilku miejscach.

#### Przykłady użycia

1. `.for_context(ctx)` - filtrowanie na podstawie przekazanego kontekstu

<code>
from django.db import models
from users.models import User
</code>

<code>
class NewEmployeeQuerySet(models.QuerySet):
    def for_user(self, user: User):
        queryset = self.filter(status="active")
        if not user.is_staff:
            queryset = queryset.filter(user=user)
        return queryset
</code>

2. `.is_CONDITION(cond?)`, `.not_CONDITION(cond?)`, `.exclude_CONDITION(cond?)` - filtrowanie na podstawie warunku

<code>
from django.db import models
</code>

<code>
class NewEmployeeQuerySet(models.QuerySet):
    def is_external(self): ...
    def exclude_franchise(self): ...
    def is_employed_for_year(self, year: int): ...
</code>

3. `.create_MODEL()`, `create_MODEL_for_CONTEXT(ctx)` - tworzenie wpisu (+ ew. side effects)

<code>
from django.contrib.auth.models import Group
from django.contrib.auth.models import UserManager

from club.models import Club
</code>

<code>
class NewEmployeeManager(UserManager):
    def create_user_for_club(self, club: Club, **kwargs):
        user = self.model.objects.create_user(
            email=club.officer.email if club.officer else "",
            **kwargs
        )
        user.groups.add(
            Group.objects.get(name="Club Users")
        )
</code>

4. `.within_RANGE()` - filtrowanie na podstawie zakresu numerycznego lub dat

<code>
import datetime
from django.db import models
from django.utils import timezone
</code>

<code>
class NewEmployeeManager(models.QuerySet):
    def within_start_date(self, days: int):
        return self.filter(
            hire__start_date__gte=(
                timezone.localtime() - datetime.timedelta(days=days)
            ).date(),
            hire__start_date__lte=timezone.localtime().date()
        )
</code>

5. `.greater_than_VALUE()`, `.less_than_VALUE()` - filtrowanie na podstawie wartości numerycznych/dat.

6. `set_FILED()`, `toggle_FIELD()` - update pola dla wszystkich elementów queryseta

7. `with_ANNOTATION()` - dodanie informacji nie dostępnych w modelu